In [47]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is 
import sklearn
assert sklearn.__version__ >= "0.20"
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_predict
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, roc_curve, roc_auc_score 


# keras
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam

# Common imports
import numpy as np
import pandas as pd
import os

# to make this notebook's output stable across runs
np.random.seed(42)
from random import randint

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [48]:
X = np.load("smiley_X.npy")
y = np.load("smiley_y.npy")
print(X.shape)
print(y.shape)

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=0)
print(x_train.shape)
print(y_train.shape)

(144, 9, 9, 1)
(144,)
(108, 9, 9, 1)
(108,)


In [49]:
# Architecture 1

In [50]:
model = keras.models.Sequential([
  keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(9,9,1)),
  keras.layers.MaxPooling2D(2, 2),
  keras.layers.Flatten(),
  keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 7, 7, 64)          640       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 3, 3, 64)         0         
 2D)                                                             
                                                                 
 flatten_6 (Flatten)         (None, 576)               0         
                                                                 
 dense_14 (Dense)            (None, 10)                5770      
                                                                 
Total params: 6,410
Trainable params: 6,410
Non-trainable params: 0
_________________________________________________________________


In [51]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
4/4 [==============================] - 1s 4ms/step - loss: 2.3099 - accuracy: 0.0463
Epoch 2/5
4/4 [==============================] - 0s 6ms/step - loss: 2.0279 - accuracy: 0.6389
Epoch 3/5
4/4 [==============================] - 0s 7ms/step - loss: 1.7580 - accuracy: 0.7963
Epoch 4/5
4/4 [==============================] - 0s 7ms/step - loss: 1.4966 - accuracy: 0.8333
Epoch 5/5
4/4 [==============================] - 0s 6ms/step - loss: 1.2532 - accuracy: 0.8519


In [52]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test Accuracy:', score[1])

Test Loss: 1.0945379734039307
Test Accuracy: 0.8888888955116272


In [53]:
y_pred = model.predict(x_test)

2/2 [==============================] - 0s 4ms/step


In [54]:
tn, fp, fn, tp = confusion_matrix(y_test, np.argmax(y_pred, axis=1)).ravel()
print("TN: %.0f" % tn + ", FP: %.0f" % fp + ", FN: %.0f" % fn + ", TP: %.0f" % tp)
print("Precision:",precision_score(y_test, np.argmax(y_pred, axis=1)))
print("Recall:", recall_score(y_test, np.argmax(y_pred, axis=1)))
print("f1 score:", f1_score(y_test, np.argmax(y_pred, axis=1)))
print("Sensitivity:", tp / (tp + fn))
print("Specificity:", tn / (tn + fp))
print("AURC:", roc_auc_score(y_test, np.argmax(y_pred, axis=1)))

TN: 14, FP: 3, FN: 1, TP: 18
Precision: 0.8571428571428571
Recall: 0.9473684210526315
f1 score: 0.9
Sensitivity: 0.9473684210526315
Specificity: 0.8235294117647058
AURC: 0.8854489164086686


In [55]:
# Architecture 2

In [56]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(9,9,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 7, 7, 32)          320       
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 3, 3, 32)         0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 1, 1, 64)          18496     
                                                                 
 flatten_7 (Flatten)         (None, 64)                0         
                                                                 
 dense_15 (Dense)            (None, 256)               16640     
                                                                 
 dense_16 (Dense)            (None, 128)               32896     
                                                      

In [57]:
# Test Using 5 epochs and adam as the optimizer.

In [58]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
4/4 [==============================] - 1s 5ms/step - loss: 1.5298 - accuracy: 0.6204
Epoch 2/5
4/4 [==============================] - 0s 5ms/step - loss: 1.2426 - accuracy: 0.9352
Epoch 3/5
4/4 [==============================] - 0s 6ms/step - loss: 0.8498 - accuracy: 0.9259
Epoch 4/5
4/4 [==============================] - 0s 6ms/step - loss: 0.5571 - accuracy: 0.9259
Epoch 5/5
4/4 [==============================] - 0s 6ms/step - loss: 0.3965 - accuracy: 0.9815


In [59]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test Accuracy:', score[1])

Test Loss: 0.30917835235595703
Test Accuracy: 0.8888888955116272


In [60]:
y_pred = model.predict(x_test)

2/2 [==============================] - 0s 3ms/step


In [61]:
tn, fp, fn, tp = confusion_matrix(y_test, np.argmax(y_pred, axis=1)).ravel()
print("TN: %.0f" % tn + ", FP: %.0f" % fp + ", FN: %.0f" % fn + ", TP: %.0f" % tp)
print("Precision:",precision_score(y_test, np.argmax(y_pred, axis=1)))
print("Recall:", recall_score(y_test, np.argmax(y_pred, axis=1)))
print("f1 score:", f1_score(y_test, np.argmax(y_pred, axis=1)))
print("Sensitivity:", tp / (tp + fn))
print("Specificity:", tn / (tn + fp))
print("AURC:", roc_auc_score(y_test, np.argmax(y_pred, axis=1)))

TN: 13, FP: 4, FN: 0, TP: 19
Precision: 0.8260869565217391
Recall: 1.0
f1 score: 0.9047619047619047
Sensitivity: 1.0
Specificity: 0.7647058823529411
AURC: 0.8823529411764706


In [62]:
# Test Using 10 epochs and adam as the optimizer. As noticed the accuracy improves as epochs increase

In [63]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
4/4 [==============================] - 0s 5ms/step - loss: 0.2609 - accuracy: 0.9537
Epoch 2/10
4/4 [==============================] - 0s 5ms/step - loss: 0.1681 - accuracy: 0.9537
Epoch 3/10
4/4 [==============================] - 0s 6ms/step - loss: 0.0992 - accuracy: 0.9722
Epoch 4/10
4/4 [==============================] - 0s 6ms/step - loss: 0.0546 - accuracy: 1.0000
Epoch 5/10
4/4 [==============================] - 0s 6ms/step - loss: 0.0343 - accuracy: 1.0000
Epoch 6/10
4/4 [==============================] - 0s 7ms/step - loss: 0.0198 - accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 0s 6ms/step - loss: 0.0098 - accuracy: 1.0000
Epoch 8/10
4/4 [==============================] - 0s 12ms/step - loss: 0.0087 - accuracy: 1.0000
Epoch 9/10
4/4 [==============================] - 0s 8ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 10/10
4/4 [==============================] - 0s 6ms/step - loss: 0.0023 - accuracy: 1.0000


In [64]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test Accuracy:', score[1])

Test Loss: 0.0025431776884943247
Test Accuracy: 1.0


In [65]:
y_pred = model.predict(x_test)

2/2 [==============================] - 0s 3ms/step


In [66]:
tn, fp, fn, tp = confusion_matrix(y_test, np.argmax(y_pred, axis=1)).ravel()
print("TN: %.0f" % tn + ", FP: %.0f" % fp + ", FN: %.0f" % fn + ", TP: %.0f" % tp)
print("Precision:",precision_score(y_test, np.argmax(y_pred, axis=1)))
print("Recall:", recall_score(y_test, np.argmax(y_pred, axis=1)))
print("f1 score:", f1_score(y_test, np.argmax(y_pred, axis=1)))
print("Sensitivity:", tp / (tp + fn))
print("Specificity:", tn / (tn + fp))
print("AURC:", roc_auc_score(y_test, np.argmax(y_pred, axis=1)))

TN: 17, FP: 0, FN: 0, TP: 19
Precision: 1.0
Recall: 1.0
f1 score: 1.0
Sensitivity: 1.0
Specificity: 1.0
AURC: 1.0


In [67]:
# Architecture 3
# Test Using 5 epochs and adam as the optimizer with learning rate 0.0001

In [68]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(9,9,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))
keras.optimizers.Adam(
learning_rate=0.0001,
beta_1=0.9,
beta_2=0.999,
epsilon=1e-07,
amsgrad=False,
name="Adam",
    
)
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 7, 7, 32)          320       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 3, 3, 32)         0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 1, 1, 64)          18496     
                                                                 
 flatten_8 (Flatten)         (None, 64)                0         
                                                                 
 dense_18 (Dense)            (None, 256)               16640     
                                                                 
 dense_19 (Dense)            (None, 128)               32896     
                                                      

In [69]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
4/4 [==============================] - 1s 6ms/step - loss: 1.5184 - accuracy: 0.5000
Epoch 2/5
4/4 [==============================] - 0s 5ms/step - loss: 1.2324 - accuracy: 0.4907
Epoch 3/5
4/4 [==============================] - 0s 7ms/step - loss: 0.8733 - accuracy: 0.5000
Epoch 4/5
4/4 [==============================] - 0s 6ms/step - loss: 0.6082 - accuracy: 0.7963
Epoch 5/5
4/4 [==============================] - 0s 7ms/step - loss: 0.4673 - accuracy: 0.9352


In [70]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test Accuracy:', score[1])

Test Loss: 0.4013483226299286
Test Accuracy: 0.8888888955116272


In [71]:
y_pred = model.predict(x_test)

2/2 [==============================] - 0s 4ms/step


In [72]:
tn, fp, fn, tp = confusion_matrix(y_test, np.argmax(y_pred, axis=1)).ravel()
print("TN: %.0f" % tn + ", FP: %.0f" % fp + ", FN: %.0f" % fn + ", TP: %.0f" % tp)
print("Precision:",precision_score(y_test, np.argmax(y_pred, axis=1)))
print("Recall:", recall_score(y_test, np.argmax(y_pred, axis=1)))
print("f1 score:", f1_score(y_test, np.argmax(y_pred, axis=1)))
print("Sensitivity:", tp / (tp + fn))
print("Specificity:", tn / (tn + fp))
print("AURC:", roc_auc_score(y_test, np.argmax(y_pred, axis=1)))

TN: 13, FP: 4, FN: 0, TP: 19
Precision: 0.8260869565217391
Recall: 1.0
f1 score: 0.9047619047619047
Sensitivity: 1.0
Specificity: 0.7647058823529411
AURC: 0.8823529411764706


In [73]:
# Architecture 4
# Test Using 5 epochs and adam as the optimizer with learning rate 1

In [74]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(9,9,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))
keras.optimizers.Adam(
learning_rate=1,
beta_1=0.9,
beta_2=0.999,
epsilon=1e-07,
amsgrad=False,
name="Adam",

)

model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 7, 7, 32)          320       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 3, 3, 32)         0         
 2D)                                                             
                                                                 
 conv2d_16 (Conv2D)          (None, 1, 1, 64)          18496     
                                                                 
 flatten_9 (Flatten)         (None, 64)                0         
                                                                 
 dense_21 (Dense)            (None, 256)               16640     
                                                                 
 dense_22 (Dense)            (None, 128)               32896     
                                                      

In [75]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
4/4 [==============================] - 1s 5ms/step - loss: 1.5596 - accuracy: 0.3796
Epoch 2/5
4/4 [==============================] - 0s 5ms/step - loss: 1.3121 - accuracy: 0.6204
Epoch 3/5
4/4 [==============================] - 0s 5ms/step - loss: 0.9583 - accuracy: 0.5185
Epoch 4/5
4/4 [==============================] - 0s 6ms/step - loss: 0.6315 - accuracy: 0.8519
Epoch 5/5
4/4 [==============================] - 0s 7ms/step - loss: 0.4533 - accuracy: 0.9630


In [76]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test Accuracy:', score[1])

Test Loss: 0.3703611195087433
Test Accuracy: 0.8888888955116272


In [77]:
y_pred = model.predict(x_test)

2/2 [==============================] - 0s 3ms/step


In [78]:
# We can notice increase in accuracy on increasing learning rate.

In [79]:
tn, fp, fn, tp = confusion_matrix(y_test, np.argmax(y_pred, axis=1)).ravel()
print("TN: %.0f" % tn + ", FP: %.0f" % fp + ", FN: %.0f" % fn + ", TP: %.0f" % tp)
print("Precision:",precision_score(y_test, np.argmax(y_pred, axis=1)))
print("Recall:", recall_score(y_test, np.argmax(y_pred, axis=1)))
print("f1 score:", f1_score(y_test, np.argmax(y_pred, axis=1)))
print("Sensitivity:", tp / (tp + fn))
print("Specificity:", tn / (tn + fp))
print("AURC:", roc_auc_score(y_test, np.argmax(y_pred, axis=1)))

TN: 13, FP: 4, FN: 0, TP: 19
Precision: 0.8260869565217391
Recall: 1.0
f1 score: 0.9047619047619047
Sensitivity: 1.0
Specificity: 0.7647058823529411
AURC: 0.8823529411764706


In [80]:
# Architecture 5
# Test using the RMSprop optimizer

In [81]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(9,9,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))
keras.optimizers.RMSprop(
learning_rate=0.001,
rho=0.9,
momentum=0.1,
epsilon=1e-07,
centered=False,
name="RMSprop",
)
model.compile(optimizer='RMSprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 7, 7, 32)          320       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 3, 3, 32)         0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 1, 1, 64)          18496     
                                                                 
 flatten_10 (Flatten)        (None, 64)                0         
                                                                 
 dense_24 (Dense)            (None, 256)               16640     
                                                                 
 dense_25 (Dense)            (None, 128)               32896     
                                                     

In [82]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
4/4 [==============================] - 1s 5ms/step - loss: 1.3367 - accuracy: 0.7315
Epoch 2/5
4/4 [==============================] - 0s 6ms/step - loss: 0.7364 - accuracy: 0.5278
Epoch 3/5
4/4 [==============================] - 0s 6ms/step - loss: 0.5809 - accuracy: 0.7037
Epoch 4/5
4/4 [==============================] - 0s 6ms/step - loss: 0.3981 - accuracy: 0.9722
Epoch 5/5
4/4 [==============================] - 0s 6ms/step - loss: 0.2658 - accuracy: 0.9537


In [83]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss :', score[0])
print('Test Accuracy :', score[1])

Test Loss : 0.24237285554409027
Test Accuracy : 1.0


In [84]:
y_pred = model.predict(x_test)

2/2 [==============================] - 0s 3ms/step


In [85]:
tn, fp, fn, tp = confusion_matrix(y_test, np.argmax(y_pred, axis=1)).ravel()
print("TN: %.0f" % tn + ", FP: %.0f" % fp + ", FN: %.0f" % fn + ", TP: %.0f" % tp)
print("Precision:",precision_score(y_test, np.argmax(y_pred, axis=1)))
print("Recall:", recall_score(y_test, np.argmax(y_pred, axis=1)))
print("f1 score:", f1_score(y_test, np.argmax(y_pred, axis=1)))
print("Sensitivity:", tp / (tp + fn))
print("Specificity:", tn / (tn + fp))
print("AURC:", roc_auc_score(y_test, np.argmax(y_pred, axis=1)))

TN: 17, FP: 0, FN: 0, TP: 19
Precision: 1.0
Recall: 1.0
f1 score: 1.0
Sensitivity: 1.0
Specificity: 1.0
AURC: 1.0


In [86]:
# Architecture 6
# Test using Adadelta optimizer

In [87]:
model = keras.models.Sequential([
  keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(9,9,1)),
  keras.layers.MaxPooling2D(2, 2),
  keras.layers.Flatten(),
  
 keras.layers.Dense(5, activation='softmax')
    
])

keras.optimizers.Adadelta(
    learning_rate=0.001, rho=0.95, epsilon=1e-07, name="Adadelta")
    
    

model.compile(optimizer='Adadelta', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 7, 7, 32)          320       
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 3, 3, 32)         0         
 g2D)                                                            
                                                                 
 flatten_11 (Flatten)        (None, 288)               0         
                                                                 
 dense_27 (Dense)            (None, 5)                 1445      
                                                                 
Total params: 1,765
Trainable params: 1,765
Non-trainable params: 0
_________________________________________________________________


In [88]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
4/4 [==============================] - 1s 5ms/step - loss: 1.6157 - accuracy: 0.5093
Epoch 2/5
4/4 [==============================] - 0s 5ms/step - loss: 1.6155 - accuracy: 0.5093
Epoch 3/5
4/4 [==============================] - 0s 6ms/step - loss: 1.6153 - accuracy: 0.5093
Epoch 4/5
4/4 [==============================] - 0s 6ms/step - loss: 1.6150 - accuracy: 0.5093
Epoch 5/5
4/4 [==============================] - 0s 6ms/step - loss: 1.6148 - accuracy: 0.5093


In [89]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss :', score[0])
print('Test Accuracy :', score[1])

Test Loss : 1.6225064992904663
Test Accuracy : 0.4722222089767456


In [90]:
# Adadelta comes out as the worst in the tests

In [91]:
y_pred = model.predict(x_test)

2/2 [==============================] - 0s 3ms/step


In [93]:
tn, fp, fn, tp = confusion_matrix(y_test, np.argmax(y_pred, axis=1)).ravel()
print("TN: %.0f" % tn + ", FP: %.0f" % fp + ", FN: %.0f" % fn + ", TP: %.0f" % tp)
print("Precision:",precision_score(y_test, np.argmax(y_pred, axis=1)))
print("Recall:", recall_score(y_test, np.argmax(y_pred, axis=1)))
print("f1 score:", f1_score(y_test, np.argmax(y_pred, axis=1)))
print("Sensitivity:", tp / (tp + fn))
print("Specificity:", tn / (tn + fp))
print("AURC:", roc_auc_score(y_test, np.argmax(y_pred, axis=1)))

TN: 17, FP: 0, FN: 19, TP: 0
Precision: 0.0
Recall: 0.0
f1 score: 0.0
Sensitivity: 0.0
Specificity: 1.0
AURC: 0.5


/home/lia/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
